## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-06-14-52-03 +0000,nypost,Resistance group members arrested in London fo...,https://nypost.com/2025/12/06/world-news/four-...
1,2025-12-06-14-52-01 +0000,nypost,Afghan man accused of supporting ISIS held top...,https://nypost.com/2025/12/06/us-news/afghan-m...
2,2025-12-06-14-48-41 +0000,nyt,"Chuck Kesey, Probiotic Yogurt Pioneer, Dies at 87",https://www.nytimes.com/2025/12/05/dining/chuc...
3,2025-12-06-14-47-17 +0000,nypost,Taking a bite out of crime: New NYPD calendar ...,https://nypost.com/2025/12/06/us-news/new-nypd...
4,2025-12-06-14-46-01 +0000,bbc,Scottish Conservative peer defects to Reform UK,https://www.bbc.com/news/articles/c5yq9yr31mgo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
139,trump,45
122,netflix,18
29,new,17
124,warner,16
334,court,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
213,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...,100
286,2025-12-05-15-33-01 +0000,nypost,Trump national security strategy calls out ‘an...,https://nypost.com/2025/12/05/us-news/trump-na...,97
246,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...,93
198,2025-12-05-19-51-16 +0000,cbc,U.S. Supreme Court to decide whether Trump's b...,https://www.cbc.ca/news/world/supreme-court-tr...,89
204,2025-12-05-19-33-39 +0000,nypost,Supreme Court to decide whether or not Trump’s...,https://nypost.com/2025/12/05/us-news/supreme-...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
213,100,2025-12-05-19-14-25 +0000,latimes,Supreme Court will hear Trump's plan to restri...,https://www.latimes.com/politics/story/2025-12...
250,73,2025-12-05-17-40-40 +0000,nypost,Here’s everything you need to know about the N...,https://nypost.com/2025/12/05/business/heres-e...
264,62,2025-12-05-16-51-04 +0000,nypost,DC pipe-bomb suspect Brian Cole Jr. appears in...,https://nypost.com/2025/12/05/us-news/accused-...
265,48,2025-12-05-16-44-00 +0000,wsj,New York Times Escalates Battle Against Perple...,https://www.wsj.com/business/media/new-york-ti...
11,42,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
96,41,2025-12-06-02-00-00 +0000,wsj,Takeaways from Robert F. Kennedy Jr.’s vaccine...,https://www.wsj.com/politics/skeptics-in-the-s...
28,39,2025-12-06-12-52-57 +0000,nypost,National Guard shooting suspect’s close milita...,https://nypost.com/2025/12/06/us-news/national...
12,39,2025-12-06-14-31-08 +0000,cbc,Russia unleashes massive drone and missile att...,https://www.cbc.ca/news/world/russia-ukraine-a...
246,36,2025-12-05-17-49-24 +0000,nypost,President Trump honored as ‘very first winner’...,https://nypost.com/2025/12/05/us-news/presiden...
166,35,2025-12-05-21-47-07 +0000,nyt,Video of Boat Strike Shows Survivors Waving Be...,https://www.nytimes.com/2025/12/05/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
